# Test CMIP6 Download

1. Confirm all four variables have been downloaded for all experiments
2. Confirm downloaded variables all cover the same time ranges for all experiments
3. Confirm calculated indices cover the same time range

In [1]:
import sys
sys.path.append('../')

#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model


In [2]:
import xarray
import numpy
import matplotlib.pyplot as plt

In [3]:
import warnings
warnings.filterwarnings('ignore')

These are the experiments

In [4]:
deckSet=['piControl','historical']
scenarioSet=[#'ssp126', 'ssp245', 'ssp370',
    'ssp585']
experimentSet=[*deckSet, *scenarioSet]

These are the models

In [5]:
modelSet=_model.scenarioMip

In [6]:
# for all models ....
for i in range(0,len(modelSet)):
  # everyone loves nested for loops  
    iModel=modelSet[i]
    print(str(i) + str(iModel))
    
    #for all experiments in that model ...
    for experiment in experimentSet: 
        
        #choose the right variant field
        if experiment=='piControl':
            variant = iModel[2]
        else:
            variant = iModel[3]

        try:
            #calculated indeces
            indecesDs = xarray.open_dataset(
                '../results/cmipWarmSeasonIndeces/' + iModel[1] +'_'+ experiment + '.nc')

            calc=[indecesDs.year.values[0], #start year
                indecesDs.year.values[-1]+1] #end year
            
            #for all four variables
            for variable in ['ts_Amon', 'psl_Amon','pr_Amon', 'tas_Amon', 'tos_Omon']:
                
                #open the files
                sourceXr = fh.loadModelData(iModel[1], variable, experiment, variant)
                
                #start year, end year for each experiment
                if experiment=='piControl':
                    source=[sourceXr.time.dt.year.values[0],
                        sourceXr.time.dt.year.values[-1]]
                elif experiment=='historical':
                    histStart=sourceXr.time.dt.year.values[0]
                    source=[histStart,
                        sourceXr.time.dt.year.values[-1]]
                else:
                    #the remaining experiments are the scenarios,
                    #where indices start in the historical dataset 
                    #and move to the scenario part way through
                    source=[histStart,
                        sourceXr.time.dt.year.values[-1]]
                
                #compare start year and end year for this variable to the indeces calculated
                if not(numpy.array_equal(source,calc)):
                    print('noMatch' + experiment + variable)
                    print([*calc, *source])
            
            #check how long the piControl is
            #(should be 500 years per CMIP specs, 
            #but some models appear not to have recalculated for all variants)
            if all(
            [calc[1]-calc[0]<400, 
             experiment=='piControl']
            ):
                print('piControl too short')
                print([*calc, *source])
        
        
        except Exception as e:
            print(experiment + ':')
            print(e)
        
        

0['CSIRO-ARCCSS' 'ACCESS-CM2' 'r1i1p1f1_gn' 'r1i1p1f1_gn']
1['CSIRO' 'ACCESS-ESM1-5' 'r1i1p1f1_gn' 'r1i1p1f1_gn']
2['AWI' 'AWI-CM-1-1-MR' 'r1i1p1f1' 'r1i1p1f1']
3['BCC' 'BCC-CSM2-MR' 'r1i1p1f1' 'r1i1p1f1']
4['CAMS' 'CAMS-CSM1-0' 'r1i1p1f1' 'r1i1p1f1']
5['CAS' 'CAS-ESM2-0' 'r1i1p1f1' 'r1i1p1f1']
noMatchpiControltos_Omon
[1, 549, 1, 499]
6['NCAR' 'CESM2' 'r1i1p1f1' 'r10i1p1f1']
7['NCAR' 'CESM2-WACCM' 'r1i1p1f1' 'r1i1p1f1']
8['THU' 'CIESM' 'r1i1p1f1' 'r1i1p1f1']
9['CMCC' 'CMCC-CM2-SR5' 'r1i1p1f1' 'r1i1p1f1']
10['CMCC' 'CMCC-ESM2' 'r1i1p1f1' 'r1i1p1f1']
11['CNRM-CERFACS' 'CNRM-CM6-1' 'r1i1p1f2' 'r1i1p1f2']
12['CNRM-CERFACS' 'CNRM-ESM2-1' 'r1i1p1f2' 'r1i1p1f2']
13['CCCma' 'CanESM5' 'r1i1p1f1' 'r1i1p1f1']
14['CCCma' 'CanESM5-CanOE' 'r1i1p2f1' 'r1i1p2f1']
15['EC-Earth-Consortium' 'EC-Earth3' 'r1i1p1f1' 'r1i1p1f1']
16['EC-Earth-Consortium' 'EC-Earth3-CC' 'r1i1p1f1' 'r1i1p1f1']
17['EC-Earth-Consortium' 'EC-Earth3-Veg' 'r1i1p1f1' 'r1i1p1f1']
noMatchhistoricalts_Amon
[1850, 2014, 1851, 2014]
18['

Those errors all look ok:

3: BCC - the source file has both a latitude and lat, so it gives this warning but is ok.

5: Not really sure why times are different between the two, but there you go.
